In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from ipywidgets import Layout
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import os
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.tools import YouTubeSearchTool
from langchain.document_loaders import YoutubeLoader
import ast 

# Define constants
CONTEXT = "context"
HUMAN_INPUT = "human_input"
CHAT_HISTORY_INDICATOR = "chat_history_indicator"
OPENAI_CHAT_MODEL = "gpt-3.5-turbo-16k"
# OPENAI_API_KEY = os.environ.get(OPENAIAPIKEY)


script_chain = None

def return_generate_ai_script(template_text: str, user_query: str, all_docs: list) -> str:
    """
    Generate an AI script and append it to a template.

    Args:
        template_text (str): Template for LLM model.
        message_text (str): User Question.
        retrieved_docs (list): List of retrieved documents.

    Returns:
        str: AI output string.
    """
    global script_chain

    # Load an embedding model from OpenAI
    embeddings = OpenAIEmbeddings()

    # Transform my docs into vectors and store that into a database (chroma) for managing and querying the embeddings
    docsearch = Chroma.from_texts(all_docs, embeddings, metadatas=[{"source": i} for i in range(len(all_docs))])

    # Use cosign similarity to perform the search for documents similar to the user query
    similar_docs = docsearch.similarity_search(user_query)
    
    # Initialize script_chain if it doesn't exist
    if script_chain is None:
             
        # Input for the prompt
        prompt = PromptTemplate(input_variables=[CHAT_HISTORY_INDICATOR, HUMAN_INPUT, CONTEXT], template=template_text)
    
        # Input for the Memory class
        memory = ConversationBufferMemory(memory_key=CHAT_HISTORY_INDICATOR, input_key = HUMAN_INPUT )

        # Load LLM model
        llm = ChatOpenAI(model_name=OPENAI_CHAT_MODEL, temperature=0)        

        # Feed LLM model, memory object, and prompt to the Q and A chain library
        script_chain = load_qa_chain(llm = llm, chain_type="stuff", memory= memory, prompt=prompt)
        
    gen_ai_output = script_chain({"input_documents": similar_docs, HUMAN_INPUT: user_query}, return_only_outputs=True)

    print('Chain memory: ', script_chain.memory.buffer)

    return gen_ai_output['output_text']


def return_youtube_docs(query, num_of_docs):
    tool = YouTubeSearchTool()
    search_result = tool.run(query + ', '+ str(num_of_docs))
    result_list = ast.literal_eval(search_result)

    all_docs = []
    
    for link in result_list:
    
        loader = YoutubeLoader.from_youtube_url(
            'https://www.youtube.com' + link, add_video_info=True
        )
    
        doc = loader.load()
        all_docs.append(doc[0].page_content)

    return all_docs